# DS 1 - LABORATORIO 05

### Juan Lorthiois       -  18771
### Roberto Castillo    -  
### Alexis Estrada       -  

#### Incisos 1 a 3:

A continuacion se importan los datos, y se inicia con la limpieza de los mismos en preparación apra el EDA y la aplicacion del agoritmo clasificador.

#### Importacion de librerías e importación de los datos como DataFrame de Pandas

In [64]:
# Importando librerias

import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

import numpy as np
import re

from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jlort\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jlort\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
#Importando los datos
Df = pd.read_csv("train.csv")

In [128]:
Df.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [129]:
Df.isna().sum()  #encontrar columnas que tienen Nas.

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [130]:
Df.tail()

,id,keyword,location,text,target
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1
7612,10873,NaN,NaN,The Latest: More Homes Razed by Northern Calif...,1


### Ideas preliminares:

Acabamos de ver que hay Nas, sin embargo la mayorí se concentran en a variable "location". Antes de decidir cómo resolver este inconveniente, se trabajará primero en limpiar los "tweets" mostrados en la variable "text". Posteriormente a este data cleaning, se procederá a decidir si se preservan o no las variables "location" y "keyword".

#### Limpieza de Tweets

In [131]:
#preparamos una funcion que se encargue de limpiar cada tweet

def Tweet_prep(linea):
    
    linea = linea.lower() #poner todo en minusculas
    linea = re.sub(r'http\S+', '', linea) #quitar urls
    linea = re.sub(r"[^a-zA-Z 911]", "", linea) #quitar cualquier cosa que no sean letras o el 911 (tel de emergencias en USA)
    linea = re.sub(' +', ' ',linea) #eliminando cualquier espacio doble o más
    tokens= word_tokenize(linea) #Separamos el String en sus distintas palabras y las ingresamos a un listado de 'Tokens'.
    nuevos_tokens = []
    for i in tokens:
        if i not in stop_words:
            nuevos_tokens.append(i)
    tokens = nuevos_tokens
    
    return tokens

In [132]:
Tweets_listos = [Tweet_prep(linea) for linea in Df.loc[:, 'text']]  #La lista de listas de tweets listos para ser reasignados al Df

In [133]:
len(Df.loc[:, 'text']) #Verificando que las listas tengan el mismo tamaño

7613

In [134]:
len(Tweets_listos)

7613

In [135]:
for i in range(0, len(Tweets_listos)):    #Agregamos las listas de palabras procesadas y listas para su analisis
    Df.at[i, 'text'] = Tweets_listos[i]

In [136]:
Df.head(10)  #listo el Df con los tweets procesados!

,id,keyword,location,text,target
0,1,NaN,NaN,"[deeds, reason, earthquake, may, allah, forgiv...",1
1,4,NaN,NaN,"[forest, fire, near, la, ronge, sask, canada]",1
2,5,NaN,NaN,"[residents, asked, shelter, place, notified, o...",1
3,6,NaN,NaN,"[1, people, receive, wildfires, evacuation, or...",1
4,7,NaN,NaN,"[got, sent, photo, ruby, alaska, smoke, wildfi...",1
5,8,NaN,NaN,"[rockyfire, update, california, hwy, closed, d...",1
6,10,NaN,NaN,"[flood, disaster, heavy, rain, causes, flash, ...",1
7,13,NaN,NaN,"[im, top, hill, see, fire, woods]",1
8,14,NaN,NaN,"[theres, emergency, evacuation, happening, bui...",1
9,15,NaN,NaN,"[im, afraid, tornado, coming, area]",1


#### Frecuencias de palabras y bigramas y trigramas

A continuacion evaluamos las palabras mas frecuentes en el Df general, pero tambien diferenciando entre tweets que se refieren a desastres reales y los que no.

Además, para el análisis en bigramas y trigramas procedemos de manera similar a lo indicado en la pagina: https://towardsdatascience.com/text-analysis-basics-in-python-443282942ec5

In [149]:
#Separamos el dataframe en desastres reales y falsos para uso futuro

criterio = Df['target'] == 0

Df_no_desastres = Df[criterio]

Df_solo_desastres = Df[~criterio]


In [157]:
#Ahora contamos la ocurrencia de palabras en el Df, Df_no_desastres, y Df_solo_desastres

Frecuencias_Df = pd.value_counts(np.hstack(Df['text']))
Frecuencias_Df_no_desastres = pd.value_counts(np.hstack(Df_no_desastres['text']))
Frecuencias_Df_solo_desastres = pd.value_counts(np.hstack(Df_solo_desastres['text']))

In [158]:
Frecuencias_Df_solo_desastres

1             295
fire          179
news          133
via           121
disaster      116
             ... 
riveeeeeer      1
iiii            1
swollen         1
idiots          1
flip            1
Length: 8903, dtype: int64

### Analisis Exploratorio

### Entrenamiento del modelo